## Explore Board Representations

In [1]:
import numpy as np

## Ideas
* cribbing this from [pgx](https://github.com/sotetsuk/pgx/blob/main/docs/backgammon.md)
* there are 24 points, you represent pieces by 1 for white and -1 for black
* you add to the point you move to, you subtract to the point you take away
* you evaluate legal moves by checking to see that >= -1 for white, <= 1 for black
* say white goes 0-23, black goes 23-0

In [2]:
board = np.zeros([1,24],dtype=int)

In [3]:
board

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0]])

In [4]:
board.reshape([4,6])

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [5]:
def create_initial_board():
    'intialize a board array with the starting positions of all checkers'
    initial_board = np.array([2, 0, 0, 0, 0, -5,
                       0, -3, 0, 0, 0, 5,
                       -5, 0, 0, 0, 3, 0,
                       5, 0, 0, 0, 0, -2])
    return initial_board

def roll():
    return np.random.randint(low=1, high=6, size=2)

In [6]:
initial_board = create_initial_board()

In [7]:
white_indices = np.where(initial_board > 0)
white_dict = {index: value for index, value in enumerate(initial_board[white_indices])}

In [8]:
white_dict

{0: 2, 1: 5, 2: 3, 3: 5}

In [9]:
open_spaces = np.argwhere((initial_board == 0) | (initial_board == -1))

In [10]:
def find_combinations(board, roll):

    open_spaces = np.argwhere((board == 0) | (board == -1))
    checkers = np.argwhere(board > 0)
    
    # Reshape roll and checkers to enable broadcasting
    roll = np.array(roll).reshape(-1, 1)
    checkers = np.array(checkers).reshape(1, -1)

    # Calculate all possible sums
    sums = roll + checkers

    # Initialize an empty list to store the combinations
    combinations = []

    # Loop through each element in open_spaces
    for space in open_spaces:
        # Find the indices where the sum equals the current open space
        indices = np.argwhere(sums == space)

        # For each pair of indices, append the corresponding elements from roll and checkers to the combinations list
        for i, j in indices:
            combinations.append((roll[i, 0], checkers[0, j]))

    return combinations

In [11]:
test_roll = roll()
test_roll

array([1, 5])

In [12]:
initial_moves = find_combinations(board=initial_board, 
                                  roll=test_roll
                                 )

In [13]:
initial_moves

[(1, 0), (1, 16), (1, 18), (5, 16)]

## Translate a 24 value numpy array into a visualized board state 

In [14]:
def create_checkers(position):
    drawn_points = []
    for point in position[:12]:
        if point == 0:
            drawn_point = 10 * ' '
        if point >= 0:
            drawn_point = 'X' * point + (10 - point) * ' '
        if point < 0:
            drawn_point = 'O' * abs(point) + (10 + point) * ' '
        drawn_points.append(drawn_point)
    for point in position[12:]:
        if point == 0:
            drawn_point = 10 * ' '
        if point >= 0:
            drawn_point = (10 - point) * ' ' + 'X' * point 
        if point < 0:
            drawn_point = (10 + point) * ' ' + 'O' * abs(point)  
        drawn_points.append(drawn_point)
    return drawn_points

def calculate_remaining_pips(position):
    x_pips = o_pips = 0
    for i, point in enumerate(position):
        if point > 0:
            dist = 23 - i
            x_pips += point * dist
        if point < 0:
            dist = i
            o_pips += point * dist * -1
    return x_pips, o_pips
    
def draw_board(position):
    drawn_position = create_checkers(position) 
    x_pips, o_pips = calculate_remaining_pips(position)
    
    print(f'''
     |---------------------|
  12 |{drawn_position[11]}|{drawn_position[12]}|13
  11 |{drawn_position[10]}|{drawn_position[13]}|14
  10 |{drawn_position[9]}|{drawn_position[14]}|15
   9 |{drawn_position[8]}|{drawn_position[15]}|16
   8 |{drawn_position[7]}|{drawn_position[16]}|17
   7 |{drawn_position[6]}|{drawn_position[17]}|18
     |---------------------|
     |---------------------|
   6 |{drawn_position[5]}|{drawn_position[18]}|19
   5 |{drawn_position[4]}|{drawn_position[19]}|20
   4 |{drawn_position[3]}|{drawn_position[20]}|21
   3 |{drawn_position[2]}|{drawn_position[21]}|22
   2 |{drawn_position[1]}|{drawn_position[22]}|23
   1 |{drawn_position[0]}|{drawn_position[23]}|24
     |---------------------|

    X remaining pips : {x_pips}
    O remaining pips : {o_pips}
    ''')

In [15]:
draw_board(initial_board)


     |---------------------|
  12 |XXXXX     |     OOOOO|13
  11 |          |          |14
  10 |          |          |15
   9 |          |          |16
   8 |OOO       |       XXX|17
   7 |          |          |18
     |---------------------|
     |---------------------|
   6 |OOOOO     |     XXXXX|19
   5 |          |          |20
   4 |          |          |21
   3 |          |          |22
   2 |          |          |23
   1 |XX        |        OO|24
     |---------------------|

    X remaining pips : 152
    O remaining pips : 152
    


## TODO
- make function to show all legal move options in normal notation
- make function to update board from move
- add in the bar
- update position function to handle hits

In [16]:
# move='8/5 6/5'
# def parse_move(move, roll):
#     first_move = move.split(' ')[0]
#     second_move = move.split(' ')[1]
#     :
#     if legal:
#         return position
#     else:
#         print('Not a legal move in this position')

In [17]:
def update_position(position,roll,move):
    return position

In [18]:
def evaluate_legal_moves(board, roll, color):
    list_of_moves = []
    # find indices with checkers
    white_checkers, black_checkers = get_points_with_checkers(board)
    # find indices with 1 or fewer of opponents checkers
    
    # find all possible legal moves for each index in first list to second list
    # write these to the 6/5, 8/5 notation we understand (3,1 for first roll)
    return list_of_moves